# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.36.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

**The DataSet is for bank personal loan and its a classification task. Will try to build a model and predict if a person will apply for a loan.**

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'mlops-capstone-project'
project_folder = './mlops-capstone-project-folder'

experiment = Experiment(ws, experiment_name)
experiment


Name,Workspace,Report Page,Docs Page
mlops-capstone-project,MLOPS,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-demo-1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [9]:
found = False
key = "Bank_Loan"
description_text = "Bank Personal Loan Dataset for Udacity capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
    print('not found ... Please make sure its correct')
        


df = dataset.to_pandas_dataframe()
df.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Personal Loan",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
mlops-capstone-project,AutoML_909ec8c9-cf3a-4db3-8c51-221d37dfadba,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [22]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print('------------------------------------------------------------------------------------------------------------')
#print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)


Run(Experiment: mlops-capstone-project,
Id: AutoML_909ec8c9-cf3a-4db3-8c51-221d37dfadba_19,
Type: azureml.scriptrun,
Status: Completed)
------------------------------------------------------------------------------------------------------------
balanced_accuracy 0.9399622017061757
precision_score_weighted 0.9856248994239382
accuracy 0.9858001989037967
matthews_correlation 0.9158248328642946
f1_score_micro 0.9858001989037967
weighted_accuracy 0.9954100481345639
precision_score_micro 0.9858001989037967
norm_macro_recall 0.8799244034123516
AUC_macro 0.99753676748537
recall_score_macro 0.9399622017061757
f1_score_macro 0.9573518545896004
recall_score_micro 0.9858001989037967
average_precision_score_micro 0.999002912847805
f1_score_weighted 0.9855165876365009
precision_score_macro 0.9765967481798886
average_precision_score_weighted 0.997976467263682
recall_score_weighted 0.9858001989037967
average_precision_score_macro 0.9905092556227341
AUC_weighted 0.99753676748537
AUC_micro 0.99898501909

In [25]:
#TODO Save the best model
bestModel = best_run.register_model(model_path='outputs/model.pkl', model_name='CapstoneModel_automl',
                        tags={'Capstone AML Training tag':'Capstone Auto ML'},
                        properties={'Accuracy': best_run_metrics['AUC_weighted']})

print(bestModel)

Model(workspace=Workspace.create(name='MLOPS', subscription_id='1db20fb5-44d4-452c-9f69-3e242db6cb9e', resource_group='ml-resource-group'), name=CapstoneModel_automl, id=CapstoneModel_automl:1, version=1, tags={'Capstone AML Training tag': 'Capstone Auto ML'}, properties={'Accuracy': '0.99753676748537'})


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [36]:
from azureml.core import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoringScript.py')
#best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')
inference_config = InferenceConfig(entry_script='scoringScript.py',
                                    environment=best_run.get_environment())
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1) # To Deploy the model to ACI with a default environment you can still customize the deploy configuration (i.e. the number of cores and amount of memory made available for the deployment) using the AciWebservice.deploy_configuration(). (https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.aci.aciwebservice?view=azure-ml-py)
service = Model.deploy(ws, "joydeep-capstone-endpoint", [bestModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-05 01:40:06+00:00 Creating Container Registry if not exists.
2022-01-05 01:40:06+00:00 Registering the environment.
2022-01-05 01:40:06+00:00 Use the existing image.
2022-01-05 01:40:06+00:00 Generating deployment configuration.
2022-01-05 01:40:07+00:00 Submitting deployment to compute.
2022-01-05 01:40:10+00:00 Checking the status of deployment joydeep-capstone-endpoint..
2022-01-05 01:40:45+00:00 Checking the status of inference endpoint joydeep-capstone-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://f40267f5-2c3a-42ef-93db-8febef8c4df0.eastus2.azurecontainer.io/score
http://f40267f5-2c3a-42ef-93db-8febef8c4df0.eastus2.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [34]:
import requests

test_data = '{"data":[{"ID":19,"Age":46,"Experience":21,"Income":193,"ZIP Code":91604,"Family":2,"CCAvg":8.1,'\
'"Education":3,"Mortgage":0,"Securities Account":0,"CD Account":0,"Online":0,"CreditCard":0}],"method":"predict"}'
print(test_data)
headers = {'Content-type': 'application/json'}

response = requests.post(service.scoring_uri, test_data, headers=headers)
print("---------------------------Predicted output (1: Loan Approved , 0: Loan Denied)-------------------------")
print(response.text)

{"data":[{"ID":19,"Age":46,"Experience":21,"Income":193,"ZIP Code":91604,"Family":2,"CCAvg":8.1,"Education":3,"Mortgage":0,"Securities Account":0,"CD Account":0,"Online":0,"CreditCard":0}],"method":"predict"}
---------------------------Predicted output (1: Loan Approved , 0: Loan Denied)-------------------------
"{\"result\": [1]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [35]:
print(service.get_logs())
service.delete()

2022-01-05T00:56:38,432466200+00:00 - iot-server/run 
2022-01-05T00:56:38,435915300+00:00 - rsyslog/run 
2022-01-05T00:56:38,446732300+00:00 - nginx/run 
2022-01-05T00:56:38,461196900+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
rsyslogd: /azureml-envs/azureml_84c85d362f11658b9008714e1aa4657b/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-01-05T00:56:38,785562300+00:00 - iot-server/finish 1 0
2022-01-05T00:56:38,789343900+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (72)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 100
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2022-01-05 00:56:42,013 | root | INFO | Starting up app insights client
logging socket was

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
